<a href="https://colab.research.google.com/github/michelmalki/MichelMalki/blob/main/TransportationProblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y -qq glpk-utils # installing a solver FIRST

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121747 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [ ]:
!pip install -q pyomo # install pyomo SECOND to find that solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.7 MB/s eta 0:00:00


In [ ]:
from pyomo.environ import *  # as a third step run the code

# Concrete Model version of the Transportation Problem

model = ConcreteModel(name="(TP)")
N= [1,2]
M= [1,2,3]
t={(1,1):10,
   (1,2):20,
   (1,3):15,
   (2,1):12,
   (2,2):15,
   (2,3):18}
C=[500,400]
D=[400,200,300]
# declare decision variables
model.x = Var(N,M,domain=NonNegativeReals)

# declare objective
def obj_rule(mdl):
  return sum(t[n,m]*model.x[n,m] for n in N for m in M)
model.obj = Objective(rule=obj_rule)

# declare constraints
def origin_rule(mdl,n):
  return sum(model.x[n,m] for m in M) <= C[n-1]
model.origin = Constraint(N,rule=origin_rule)
def destination_rule(mdl,m):
  return sum (model.x[n,m] for n in N) == D[m-1]
model.destination = Constraint(M,rule=destination_rule)

# solve
results = SolverFactory('glpk').solve(model)
results.write()
if results.solver.status:
    model.pprint()

# display solution
print('\nCost = ', model.obj())

print('\nDecision Variables')
for n in N:
    for m in M:
        print(f'x[{n},{m}] = ', model.x[n,m].value)

print('\nConstraints')
print('Origin Points  = ', [model.origin[n]() for n in N])
print('Destination Points = ', [model.destination[m]() for m in M])

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 11900.0
  Upper bound: 11900.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 6
  Number of nonzeros: 12
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.005948305130004883
# ----------------------------------------------------------
#   Solution Information
# -------------------------------